1. Интегрирование. Методы прямоугольников, трапеций, Симпсона. Реализовать вычисление по количеству разбиений отрезка (n) и по достигаемой точности (eps) по методу Рунге.

## Численное интегрирование

$\int_a^b f(x) dx \qquad f(x) \in [a,b]$

$\int_a^b f(x) dx = \sum_{k=1}^n A_k f(x_k)$ -- квадратурная формула $x_k \in [a,b]$

$A_k = \int_a^b l_k dx \qquad l_k$ -- коэффициент Лагранжа

$A_k = \int_a^b \frac {w(x)}{(x-x_k)w'(x_k)} dx$

$w(x) = \prod_{k=1}^n (x-x_k)$

$d$ -- целое натуральное, называется алгебраической степенью точности, если квадратурная формула точна для всех полиномов степени не выше $d$ и не точна для полиномов степени $d + 1$

### Квадратурные формулы прямоугольников.

#### Левых:

$\int_a^b f(x) dx = (b-a)f(a) \qquad d = 0$

#### Правых:

$\int_a^b f(x) dx = (b-a)f(b) \qquad d = 0$

#### Серединных:

$\int_a^b f(x) dx = (b-a)f(\frac{a+b}{2}) \qquad d = 1$

### Составная квадратурная формула.

$h = \frac {b-a}{N}, \, x_k= a+ k h, \, f_k=f(x_k), \, k = 0, \dots , N$

$\int_a^b f(x) dx = h \cdot \sum_{k=1}^{N} f(\delta + (k-1)h)$

$\delta = a $ -- левых прям.

$\delta = a+\frac{h}{2} $ -- средних.

$\delta = a + h $ -- правых прям.

### Квадратурная формула трапеций.

#### простая

$\int_a^b f(x) dx = \frac {b-a}{2} (f(a) + f(b)) \qquad d = 1$

#### составная

$\int_a^b f(x) dx = \frac {b-a}{2N} (f_0 + 2 (f_1 + \dots + f_{N-1}) + f_N)$

### Квадратурная формула Симпсона

#### простая

$\int_a^b f(x) dx = \frac {b-a}{6} (f(a) + 4f(\frac{a+b}{2}) + f(b)) \qquad d = 3 $

#### составная

$h=\frac{b-a}{2N} \qquad [x_{k-1}, x_{k+1}] \, k = 0, \dots , N$

$\int_a^b f(x) dx = \frac {b-a}{6N} (f_0 + 4(f_1 + f_3 + \cdots + f_{2N-1}) + 2(f_2 + f_4 + \cdots + f_{2N-2}) + f_{2N})$

### Оценка погрешности

$\exists f^{(d+1)}\in C[a,b]$

$| R_N (f) | = C(b-a)(\frac{b-a}{N})^{d+1} M_{d+1}$

$M_{d+1} = \max_{\xi \in [a,b]}|f^{d+1}(\xi)|$

$C = 1/2$ -- для левых и правых

$C = 1/24$ -- для средних

$C = 1/12$ -- для трапеций

$C = 1/2880$ -- для Симпсона


In [0]:
!pip install pycuda

[1024 1024 64]

In [0]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [0]:
import numpy
a = numpy.random.randn(4,4)

In [0]:
a = a.astype(numpy.float32)

In [0]:
a_gpu = cuda.mem_alloc(a.nbytes)

In [0]:
cuda.memcpy_htod(a_gpu, a)

In [0]:
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)

In [0]:
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))

In [0]:
a_doubled = numpy.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled)
print(a)

[[-1.8018398  -0.514384    2.1884437   2.5611844 ]
 [ 0.7302624   1.3989468  -0.7208217  -1.3298794 ]
 [-0.33193457  3.3238304  -0.11609285  1.1215879 ]
 [ 0.86144763 -2.038006    1.8497057  -3.8086014 ]]
[[-0.9009199  -0.257192    1.0942218   1.2805922 ]
 [ 0.3651312   0.6994734  -0.36041084 -0.6649397 ]
 [-0.16596729  1.6619152  -0.05804643  0.56079394]
 [ 0.43072382 -1.019003    0.92485285 -1.9043007 ]]
